# 🦆 Automated Exploratory Data Analysis with DuckDB

This notebook provides automated EDA for:
- **CSV files**
- **Parquet files**
- **DuckDB databases**

Just set your file path below and run all cells!


## Configuration

Set your data source path below. Supports:
- `.csv` files
- `.parquet` files  
- `.duckdb` / `.db` database files


In [5]:
# ============================================================
# 📁 SET YOUR DATA SOURCE HERE
# ============================================================

DATA_SOURCE = "/home/Alberto.Barcelos/Downloads/Crime_Data_from_2020_to_Present.csv"  # Change this to your file path

# Examples:
# DATA_SOURCE = "data/sales.csv"
# DATA_SOURCE = "data/events.parquet"
# DATA_SOURCE = "analytics.duckdb"


## Setup & Imports


In [6]:
import duckdb
from pathlib import Path
from IPython.display import display, HTML, Markdown
import warnings
warnings.filterwarnings('ignore')

def styled_header(text: str, emoji: str = "📊") -> None:
    """Display a styled section header."""
    display(HTML(f"""
    <div style="
        background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
        padding: 12px 20px;
        border-radius: 8px;
        margin: 20px 0 10px 0;
        border-left: 4px solid #e94560;
    ">
        <h3 style="color: #eee; margin: 0; font-weight: 600;">
            {emoji} {text}
        </h3>
    </div>
    """))

def styled_stat(label: str, value: str) -> None:
    """Display a styled statistic."""
    display(HTML(f"""
    <div style="
        display: inline-block;
        background: #1a1a2e;
        padding: 8px 16px;
        border-radius: 6px;
        margin: 4px;
    ">
        <span style="color: #888; font-size: 12px;">{label}</span><br>
        <span style="color: #e94560; font-size: 18px; font-weight: bold;">{value}</span>
    </div>
    """))

print("✅ Setup complete!")


✅ Setup complete!


## EDA Functions


In [7]:
class DuckDBExplorer:
    """Automated Exploratory Data Analysis with DuckDB."""
    
    def __init__(self, source: str):
        """Initialize the explorer with a data source."""
        self.source = source
        self.path = Path(source)
        self.con = None
        self.tables = []
        self._setup_connection()
    
    def _setup_connection(self) -> None:
        """Setup DuckDB connection based on source type."""
        suffix = self.path.suffix.lower()
        
        if suffix in ('.duckdb', '.db'):
            # Connect to existing DuckDB database
            self.con = duckdb.connect(str(self.path), read_only=True)
            self.tables = [row[0] for row in self.con.execute("SHOW TABLES").fetchall()]
            print(f"📂 Connected to DuckDB database: {self.path.name}")
            print(f"   Found {len(self.tables)} table(s): {', '.join(self.tables)}")
        else:
            # CSV or Parquet - create in-memory connection with view
            self.con = duckdb.connect()
            self.con.execute(f"CREATE VIEW data AS SELECT * FROM '{self.source}'")
            self.tables = ['data']
            print(f"📂 Loaded file: {self.path.name}")
    
    def overview(self, table: str = None) -> None:
        """Display high-level overview of the data."""
        tables_to_analyze = [table] if table else self.tables
        
        for tbl in tables_to_analyze:
            styled_header(f"Overview: {tbl}", "📋")
            
            # Basic counts
            row_count = self.con.execute(f"SELECT COUNT(*) FROM {tbl}").fetchone()[0]
            col_count = len(self.con.execute(f"DESCRIBE {tbl}").fetchall())
            
            display(HTML(f"""
            <div style="display: flex; gap: 20px; flex-wrap: wrap;">
                <div style="background: #1a1a2e; padding: 15px 25px; border-radius: 8px; text-align: center;">
                    <div style="color: #888; font-size: 12px;">ROWS</div>
                    <div style="color: #e94560; font-size: 24px; font-weight: bold;">{row_count:,}</div>
                </div>
                <div style="background: #1a1a2e; padding: 15px 25px; border-radius: 8px; text-align: center;">
                    <div style="color: #888; font-size: 12px;">COLUMNS</div>
                    <div style="color: #0f94d2; font-size: 24px; font-weight: bold;">{col_count}</div>
                </div>
            </div>
            """))
    
    def schema(self, table: str = None) -> None:
        """Display schema/column information."""
        tables_to_analyze = [table] if table else self.tables
        
        for tbl in tables_to_analyze:
            styled_header(f"Schema: {tbl}", "🏗️")
            schema_df = self.con.execute(f"DESCRIBE {tbl}").fetchdf()
            display(schema_df.style.set_properties(**{
                'background-color': '#1a1a2e',
                'color': '#eee',
                'border-color': '#333'
            }))
    
    def sample(self, table: str = None, n: int = 5) -> None:
        """Display sample rows."""
        tables_to_analyze = [table] if table else self.tables
        
        for tbl in tables_to_analyze:
            styled_header(f"Sample Data: {tbl} (first {n} rows)", "👀")
            sample_df = self.con.execute(f"SELECT * FROM {tbl} LIMIT {n}").fetchdf()
            display(sample_df)
    
    def summary(self, table: str = None) -> None:
        """Display statistical summary using DuckDB's SUMMARIZE."""
        tables_to_analyze = [table] if table else self.tables
        
        for tbl in tables_to_analyze:
            styled_header(f"Statistical Summary: {tbl}", "📈")
            summary_df = self.con.execute(f"SUMMARIZE {tbl}").fetchdf()
            display(summary_df)
    
    def nulls(self, table: str = None) -> None:
        """Analyze null values in each column."""
        tables_to_analyze = [table] if table else self.tables
        
        for tbl in tables_to_analyze:
            styled_header(f"Null Analysis: {tbl}", "🕳️")
            
            row_count = self.con.execute(f"SELECT COUNT(*) FROM {tbl}").fetchone()[0]
            cols = self.con.execute(f"DESCRIBE {tbl}").fetchall()
            
            null_data = []
            for col_name, col_type, *_ in cols:
                null_count = self.con.execute(
                    f'SELECT COUNT(*) FROM {tbl} WHERE "{col_name}" IS NULL'
                ).fetchone()[0]
                pct = (null_count / row_count * 100) if row_count > 0 else 0
                null_data.append({
                    'Column': col_name,
                    'Type': col_type,
                    'Null Count': null_count,
                    'Null %': f"{pct:.1f}%",
                    'Completeness': f"{100-pct:.1f}%"
                })
            
            import pandas as pd
            null_df = pd.DataFrame(null_data)
            
            # Highlight columns with nulls
            def highlight_nulls(row):
                if row['Null Count'] > 0:
                    return ['background-color: #3d1a1a'] * len(row)
                return [''] * len(row)
            
            display(null_df.style.apply(highlight_nulls, axis=1))
    
    def uniques(self, table: str = None, max_categories: int = 20) -> None:
        """Analyze unique values and cardinality."""
        tables_to_analyze = [table] if table else self.tables
        
        for tbl in tables_to_analyze:
            styled_header(f"Cardinality Analysis: {tbl}", "🔢")
            
            row_count = self.con.execute(f"SELECT COUNT(*) FROM {tbl}").fetchone()[0]
            cols = self.con.execute(f"DESCRIBE {tbl}").fetchall()
            
            cardinality_data = []
            for col_name, col_type, *_ in cols:
                unique_count = self.con.execute(
                    f'SELECT COUNT(DISTINCT "{col_name}") FROM {tbl}'
                ).fetchone()[0]
                pct = (unique_count / row_count * 100) if row_count > 0 else 0
                
                # Determine cardinality type
                if unique_count == row_count:
                    card_type = "🔑 Unique (potential key)"
                elif unique_count <= 2:
                    card_type = "🔘 Binary"
                elif unique_count <= max_categories:
                    card_type = "📊 Low (categorical)"
                else:
                    card_type = "📈 High (continuous)"
                
                cardinality_data.append({
                    'Column': col_name,
                    'Unique Values': unique_count,
                    'Uniqueness %': f"{pct:.1f}%",
                    'Type': card_type
                })
            
            import pandas as pd
            card_df = pd.DataFrame(cardinality_data)
            display(card_df)
    
    def value_counts(self, column: str, table: str = None, top_n: int = 10) -> None:
        """Show value distribution for a specific column."""
        tbl = table or self.tables[0]
        styled_header(f"Value Counts: {column} (top {top_n})", "📊")
        
        query = f'''
            SELECT "{column}" as value, COUNT(*) as count
            FROM {tbl}
            GROUP BY "{column}"
            ORDER BY count DESC
            LIMIT {top_n}
        '''
        df = self.con.execute(query).fetchdf()
        
        total = self.con.execute(f"SELECT COUNT(*) FROM {tbl}").fetchone()[0]
        df['percentage'] = (df['count'] / total * 100).round(2).astype(str) + '%'
        display(df)
    
    def correlations(self, table: str = None) -> None:
        """Calculate correlations between numeric columns."""
        tbl = table or self.tables[0]
        styled_header(f"Numeric Correlations: {tbl}", "🔗")
        
        # Get numeric columns
        cols = self.con.execute(f"DESCRIBE {tbl}").fetchall()
        numeric_cols = [
            col[0] for col in cols 
            if any(t in col[1].upper() for t in ['INT', 'FLOAT', 'DOUBLE', 'DECIMAL', 'NUMERIC', 'BIGINT'])
        ]
        
        if len(numeric_cols) < 2:
            print("⚠️ Not enough numeric columns for correlation analysis")
            return
        
        # Build correlation matrix using DuckDB
        corr_data = []
        for col1 in numeric_cols:
            row = {}
            for col2 in numeric_cols:
                corr = self.con.execute(f'''
                    SELECT CORR("{col1}", "{col2}") FROM {tbl}
                ''').fetchone()[0]
                row[col2] = round(corr, 3) if corr is not None else None
            corr_data.append(row)
        
        import pandas as pd
        corr_df = pd.DataFrame(corr_data, index=numeric_cols)
        
        # Style the correlation matrix
        def color_corr(val):
            if val is None:
                return ''
            if val > 0.7:
                return 'background-color: #1a472a; color: white'
            elif val < -0.7:
                return 'background-color: #4a1a1a; color: white'
            return ''
        
        display(corr_df.style.applymap(color_corr))
    
    def full_report(self, table: str = None) -> None:
        """Run complete EDA analysis."""
        styled_header("FULL EDA REPORT", "🦆")
        display(HTML(f"<p style='color: #888;'>Source: <code>{self.source}</code></p>"))
        
        self.overview(table)
        self.schema(table)
        self.sample(table)
        self.summary(table)
        self.nulls(table)
        self.uniques(table)
        self.correlations(table)
        
        styled_header("Analysis Complete!", "✅")
    
    def query(self, sql: str):
        """Run a custom SQL query."""
        styled_header("Custom Query", "🔍")
        result = self.con.execute(sql).fetchdf()
        display(result)
        return result
    
    def close(self) -> None:
        """Close the database connection."""
        if self.con:
            self.con.close()
            print("🔒 Connection closed")

print("✅ DuckDBExplorer class loaded!")


✅ DuckDBExplorer class loaded!


## 🚀 Run Full EDA Report

Execute the cell below to run a complete automated analysis on your data source.


In [10]:
# Initialize the explorer with your data source
eda = DuckDBExplorer(DATA_SOURCE)

# Run the full automated EDA report
eda.full_report()


📂 Loaded file: Crime_Data_from_2020_to_Present.csv


,column_name,column_type,null,key,default,extra
0,DR_NO,BIGINT,YES,None,None,None
1,Date Rptd,TIMESTAMP,YES,None,None,None
2,DATE OCC,TIMESTAMP,YES,None,None,None
3,TIME OCC,VARCHAR,YES,None,None,None
4,AREA,VARCHAR,YES,None,None,None
5,AREA NAME,VARCHAR,YES,None,None,None
6,Rpt Dist No,VARCHAR,YES,None,None,None
7,Part 1-2,BIGINT,YES,None,None,None
8,Crm Cd,BIGINT,YES,None,None,None
9,Crm Cd Desc,VARCHAR,YES,None,None,None


,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,190326475,2020-03-01,2020-03-01,2130,07,Wilshire,0784,1,510,VEHICLE - STOLEN,...,AA,Adult Arrest,510,998,<NA>,<NA>,1900 S LONGWOOD AV,None,34.0375,-118.3506
1,200106753,2020-02-09,2020-02-08,1800,01,Central,0182,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330,998,<NA>,<NA>,1000 S FLOWER ST,None,34.0444,-118.2628
2,200320258,2020-11-11,2020-11-04,1700,03,Southwest,0356,1,480,BIKE - STOLEN,...,IC,Invest Cont,480,<NA>,<NA>,<NA>,1400 W 37TH ST,None,34.0210,-118.3002
3,200907217,2023-05-10,2020-03-10,2037,09,Van Nuys,0964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,IC,Invest Cont,343,<NA>,<NA>,<NA>,14000 RIVERSIDE DR,None,34.1576,-118.4387
4,200412582,2020-09-09,2020-09-09,0630,04,Hollenbeck,0413,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510,<NA>,<NA>,<NA>,200 E AVENUE 28,None,34.0820,-118.2130


,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,DR_NO,BIGINT,817,249928885,1324249,220185515.9817868,13169630.316388726,210640509,220934962,231097187,1003448,0.00
1,Date Rptd,TIMESTAMP,2020-01-01 00:00:00,2024-12-21 00:00:00,2115,2022-05-24 10:56:48.707576,None,2021-04-23 13:15:28.402058,2022-06-06 19:48:32.66019,2023-07-01 19:46:26.733421,1003448,0.00
2,DATE OCC,TIMESTAMP,2020-01-01 00:00:00,2024-12-21 00:00:00,2115,2022-05-12 07:53:05.958814,None,2021-04-07 18:26:07.544415,2022-05-25 16:52:59.898157,2023-06-20 01:08:04.293488,1003448,0.00
3,TIME OCC,VARCHAR,0001,2359,1649,None,None,None,None,None,1003448,0.00
4,AREA,VARCHAR,01,21,21,None,None,None,None,None,1003448,0.00
5,AREA NAME,VARCHAR,77th Street,Wilshire,21,None,None,None,None,None,1003448,0.00
6,Rpt Dist No,VARCHAR,0101,2199,1805,None,None,None,None,None,1003448,0.00
7,Part 1-2,BIGINT,1,2,2,1.4005259863988966,0.4900052651739671,1,1,2,1003448,0.00
8,Crm Cd,BIGINT,110,956,148,500.2210418477091,205.3108756269528,331,442,626,1003448,0.00
9,Crm Cd Desc,VARCHAR,ARSON,WEAPONS POSSESSION/BOMBING,140,None,None,None,None,None,1003448,0.00


,Column,Type,Null Count,Null %,Completeness
0,DR_NO,BIGINT,0,0.0%,100.0%
1,Date Rptd,TIMESTAMP,0,0.0%,100.0%
2,DATE OCC,TIMESTAMP,0,0.0%,100.0%
3,TIME OCC,VARCHAR,0,0.0%,100.0%
4,AREA,VARCHAR,0,0.0%,100.0%
5,AREA NAME,VARCHAR,0,0.0%,100.0%
6,Rpt Dist No,VARCHAR,0,0.0%,100.0%
7,Part 1-2,BIGINT,0,0.0%,100.0%
8,Crm Cd,BIGINT,0,0.0%,100.0%
9,Crm Cd Desc,VARCHAR,0,0.0%,100.0%


,Column,Unique Values,Uniqueness %,Type
0,DR_NO,1003448,100.0%,🔑 Unique (potential key)
1,Date Rptd,1817,0.2%,📈 High (continuous)
2,DATE OCC,1817,0.2%,📈 High (continuous)
3,TIME OCC,1439,0.1%,📈 High (continuous)
4,AREA,21,0.0%,📈 High (continuous)
5,AREA NAME,21,0.0%,📈 High (continuous)
6,Rpt Dist No,1210,0.1%,📈 High (continuous)
7,Part 1-2,2,0.0%,🔘 Binary
8,Crm Cd,140,0.0%,📈 High (continuous)
9,Crm Cd Desc,140,0.0%,📈 High (continuous)


,DR_NO,Part 1-2,Crm Cd,Vict Age,Premis Cd,Weapon Used Cd,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LAT,LON
DR_NO,1.000000,-0.039000,-0.023000,-0.068000,0.025000,0.018000,-0.022000,0.029000,0.032000,0.152000,0.046000,-0.045000
Part 1-2,-0.039000,1.000000,0.698000,0.206000,0.280000,0.426000,0.699000,0.076000,0.009000,0.104000,-0.029000,0.029000
Crm Cd,-0.023000,0.698000,1.000000,-0.021000,0.118000,0.366000,0.999000,0.000000,0.121000,0.037000,-0.039000,0.039000
Vict Age,-0.068000,0.206000,-0.021000,1.000000,0.187000,0.082000,-0.021000,-0.023000,-0.011000,-0.004000,-0.001000,0.002000
Premis Cd,0.025000,0.280000,0.118000,0.187000,1.000000,0.205000,0.118000,-0.055000,-0.010000,-0.002000,-0.006000,0.007000
Weapon Used Cd,0.018000,0.426000,0.366000,0.082000,0.205000,1.000000,0.367000,-0.146000,-0.061000,0.029000,-0.009000,0.011000
Crm Cd 1,-0.022000,0.699000,0.999000,-0.021000,0.118000,0.367000,1.000000,0.021000,0.164000,-0.032000,-0.039000,0.039000
Crm Cd 2,0.029000,0.076000,0.000000,-0.023000,-0.055000,-0.146000,0.021000,1.000000,0.298000,0.019000,-0.015000,0.016000
Crm Cd 3,0.032000,0.009000,0.121000,-0.011000,-0.010000,-0.061000,0.164000,0.298000,1.000000,0.162000,0.020000,-0.027000
Crm Cd 4,0.152000,0.104000,0.037000,-0.004000,-0.002000,0.029000,-0.032000,0.019000,0.162000,1.000000,0.100000,-0.180000


## Individual Analysis Methods

Use these cells to run specific analyses as needed.


In [ ]:
# View value distribution for a specific column
# eda.value_counts("column_name")


In [ ]:
# Run custom SQL queries
# eda.query("SELECT * FROM data WHERE column > 100")


## Quick One-Liner Alternative

If you just need a fast summary without all the styling, use DuckDB's built-in SUMMARIZE:


In [12]:
# Quick summary using raw DuckDB (uncomment to use)
duckdb.sql(f"SUMMARIZE SELECT * FROM '{DATA_SOURCE}'").show()


┌──────────────┬─────────────┬─────────────────────────────────┬─────────────────────────────────┬───────────────┬────────────────────────────┬────────────────────┬────────────────────────────┬────────────────────────────┬────────────────────────────┬─────────┬─────────────────┐
│ column_name  │ column_type │               min               │               max               │ approx_unique │            avg             │        std         │            q25             │            q50             │            q75             │  count  │ null_percentage │
│   varchar    │   varchar   │             varchar             │             varchar             │     int64     │          varchar           │      varchar       │          varchar           │          varchar           │          varchar           │  int64  │  decimal(9,2)   │
├──────────────┼─────────────┼─────────────────────────────────┼─────────────────────────────────┼───────────────┼────────────────────────────┼─────────────────

## Cleanup


In [ ]:
# Close the connection when done (optional)
# eda.close()
